## Importing Relevant Libraries for WebScraping and Setting up the DataFrame

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### I use BS to scrape the table from wikipedia

In [3]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_ = "wikitable sortable").tbody.text

### The scraped table information is in str format. Here I convert it to a Pandas Dataframe

In [4]:
table_list = [line.split('\n') for line in (table.split('\n\n'))]

df = pd.DataFrame(table_list, columns = table_list[0])
df.drop([0,289], inplace= True)#I drop the first and last rows which are the table header and none, repectively

df.head()

,,Postcode,Borough,Neighbourhood
1,,M1A,Not assigned,Not assigned
2,,M2A,Not assigned,Not assigned
3,,M3A,North York,Parkwoods
4,,M4A,North York,Victoria Village
5,,M5A,Downtown Toronto,Harbourfront


### Table rows with unassigned Boroughs are dropped

In [5]:
na = df['Borough'] == "Not assigned"
df = df[~na].reset_index(drop = True)

In [6]:
df.shape

(211, 4)

In [7]:
df.head(10)

,,Postcode,Borough,Neighbourhood
0,,M3A,North York,Parkwoods
1,,M4A,North York,Victoria Village
2,,M5A,Downtown Toronto,Harbourfront
3,,M5A,Downtown Toronto,Regent Park
4,,M6A,North York,Lawrence Heights
5,,M6A,North York,Lawrence Manor
6,,M7A,Queen's Park,Not assigned
7,,M9A,Etobicoke,Islington Avenue
8,,M1B,Scarborough,Rouge
9,,M1B,Scarborough,Malvern


### I replace unassigned Neighborhoods with the name of the Borough

In [8]:
df['Neighbourhood'][df['Neighbourhood'] == "Not assigned"] = df['Borough'][df['Neighbourhood'] == "Not assigned"]

In [9]:
df.head(10)

,,Postcode,Borough,Neighbourhood
0,,M3A,North York,Parkwoods
1,,M4A,North York,Victoria Village
2,,M5A,Downtown Toronto,Harbourfront
3,,M5A,Downtown Toronto,Regent Park
4,,M6A,North York,Lawrence Heights
5,,M6A,North York,Lawrence Manor
6,,M7A,Queen's Park,Queen's Park
7,,M9A,Etobicoke,Islington Avenue
8,,M1B,Scarborough,Rouge
9,,M1B,Scarborough,Malvern


### Here, I aggregate the df based on Neighborhoods with similar Boroughs and Postcodes

In [10]:
df_grouped = df.groupby(['Postcode', 'Borough'], as_index = 0, sort = 0).agg(', '.join)

In [11]:
df_grouped.head()

,Postcode,Borough,,Neighbourhood
0,M3A,North York,,Parkwoods
1,M4A,North York,,Victoria Village
2,M5A,Downtown Toronto,",","Harbourfront, Regent Park"
3,M6A,North York,",","Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,,Queen's Park


#### Dropping the column with the trailing commas

In [12]:
df_grouped.drop(columns='', inplace= True)

In [13]:
df_grouped.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [14]:
df_grouped.shape

(103, 3)